In [1]:
from maps.screens import ImageScreenMultiAntibody
import json

# Set path to parameter file
param_file = "/home/kkumbier/maps/template_analyses/params/maps_multiantibody-train-multiclass.json"

PALETTE = {
    "WT": "#9A9A9A",
    "FUS": "#B24745" ,
    "C9orf72": "#6A6599",
    "sporadic": "#79AF97",
    "SOD1": "#00A1D5",
    "TDP43": "#DF8F44"
}

# Overview
This notebook shows a basic use case for the multiantibody, multiclass analysis using the MAPs framework. To handle multimarker analysis, we define a new image screen class `ImageScreenMultiAntibody`, the key difference being that it will handle loading / returning datasets from multiple antibodies simultaneously. Data preprocessing is handled as in the single marker analysis-by defining steps within the `preprocess` key of the params file-and applied independently for each loaded marker set.

In [2]:
# Load in analysis parameters
with open(param_file) as f:
    params = json.load(f)
 
print(json.dumps(params, indent=4))

# Initialize screen class and run data processing pipeline 
screen = ImageScreenMultiAntibody(params)

{
    "name": "maps_dmso",
    "screen": "20250216_AWALS37_Full_screen_n96",
    "root": "/awlab/projects/2024_ALS/Experiments",
    "data_file": "Objects_Population - Nuclei Selected.txt",
    "eval_dir": "Evaluation1",
    "preprocess": {
        "drop_na_features": {
            "na_prop": 0.1
        },
        "drop_sample_by_feature": {
            "drop_key": [
                {
                    "CellLines": [
                        "C9014",
                        "NS048"
                    ]
                }
            ]
        },
        "select_sample_by_feature": {
            "select_key": [
                {
                    "Drugs": [
                        "DMSO"
                    ],
                    "Mutations": [
                        "FUS",
                        "WT",
                        "SOD1",
                        "C9orf72"
                    ],
                    "CellLines": [
                        "NS007",
                        

The `ImageScreenMultiAntibody` class provides a function to scan data associated with the screen and list out all available markersets. This can be useful to write scripts that programatically iterate over markers without specifying them manually. For the purposes of demonstration, we select two marker sets from this list, load the associated data, and run the preprocessing steps specified in `params`.

In [3]:
print(screen.loader.list_antibodies())
screen.load(antibody=["FUS/EEA1", "COX IV/Galectin3/atubulin"])
screen.preprocess()

shape: (9,)
Series: 'Antibody' [str]
[
	"HSP70/SOD1"
	"TDP43_abcam/G3BP1"
	"CD63/SEC16A"
	"LAMP/TDP43-C"
	"pTDP43/HMOX1"
	"COX IV/Galectin3/atubulin"
	"Rab1/CHMP2B"
	"p62/LC3"
	"FUS/EEA1"
]


Data for `ImageScreenMultiAntibody` are stored as a python dict, keyed by marker set names.

In [4]:
screen.data

{'FUS/EEA1': shape: (18_154, 315)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ Total_Spo ┆ Relative_ ┆ Number_of ┆ Number_of ┆ … ┆ Membrane_ ┆ Membrane_ ┆ Membrane_ ┆ ID       │
 │ t_Area    ┆ Spot_Inte ┆ _Spots    ┆ _Spots_pe ┆   ┆ Region_Al ┆ Region_Al ┆ Region_Al ┆ ---      │
 │ ---       ┆ nsity     ┆ ---       ┆ r_Area_of ┆   ┆ exa_647_P ┆ exa_647_P ┆ exa_647_S ┆ str      │
 │ f32       ┆ ---       ┆ f32       ┆ _Ce…      ┆   ┆ rof…      ┆ rof…      ┆ ER_…      ┆          │
 │           ┆ f32       ┆           ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆          │
 │           ┆           ┆           ┆ f32       ┆   ┆ f32       ┆ f32       ┆ f32       ┆          │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ 390.0     ┆ 0.316696  ┆ 11.0      ┆ 0.008958  ┆ … ┆ 0.004415  ┆ 0.001474  ┆ 0.000482  ┆ 20240420 │
 │           ┆           ┆           ┆          

The `MAP` class is defined on a `Screen` class. When a `MAP` class is fit, it reads out analysis parameters specified by the screen (including model type and fitting strategy) and runs the corresponding analysis workflow. 

Note that models designed for single screen/marker set data are not directly applicable to multi-screen data. In particular, one needs to define how data should be integrated across screens/marker sets. We handle this using pytorch, which allows for customizable / flexible model definitions. 

In [ ]:
from maps.analyses import MAP

maps = MAP(screen)
maps.fit()

Starting cell-level training...
Cell Epoch 1/10, Overall Loss: 1.4934, Active: 2/2
  COX IV/Galectin3/atubulin - Loss: 1.5062, Acc: 0.2667 
  FUS/EEA1 - Loss: 1.4805, Acc: 0.2533 
Cell Epoch 2/10, Overall Loss: 1.3582, Active: 2/2
  COX IV/Galectin3/atubulin - Loss: 1.3351, Acc: 0.3617 
  FUS/EEA1 - Loss: 1.3812, Acc: 0.2933 
Cell Epoch 3/10, Overall Loss: 1.3745, Active: 2/2
  COX IV/Galectin3/atubulin - Loss: 1.3918, Acc: 0.3067 
  FUS/EEA1 - Loss: 1.3571, Acc: 0.3433 
Cell Epoch 4/10, Overall Loss: 1.3563, Active: 2/2
  COX IV/Galectin3/atubulin - Loss: 1.3799, Acc: 0.3467 
  FUS/EEA1 - Loss: 1.3327, Acc: 0.3717 
Cell Epoch 5/10, Overall Loss: 1.3501, Active: 2/2
  COX IV/Galectin3/atubulin - Loss: 1.3830, Acc: 0.3300 
  FUS/EEA1 - Loss: 1.3172, Acc: 0.3500 
Cell Epoch 6/10, Overall Loss: 1.3000, Active: 2/2
  COX IV/Galectin3/atubulin - Loss: 1.3366, Acc: 0.3567 
  FUS/EEA1 - Loss: 1.2633, Acc: 0.4233 
Cell Epoch 7/10, Overall Loss: 1.3010, Active: 2/2
  COX IV/Galectin3/atubulin -

KeyError: 'FUS'

In [8]:
maps.fitted["predicted"].head(24)

Class_0,Class_1,Class_2,Class_3,CellLines,True
f32,f32,f32,f32,str,i64
0.304352,0.203887,0.271258,0.220503,"""NS022""",2
0.241029,0.221847,0.296057,0.241066,"""C9012""",3
0.243928,0.356209,0.212806,0.187057,"""C9031""",3
0.192734,0.216386,0.299083,0.291797,"""NS025""",2
0.353189,0.204125,0.232498,0.210188,"""GIH91""",0
…,…,…,…,…,…
0.241837,0.312448,0.26744,0.178275,"""NS007""",1
0.235724,0.253948,0.231675,0.278652,"""NS013""",2
0.302772,0.26825,0.233833,0.195145,"""NS045""",1


Thre resulting predictions can be further processed as described in the single marker set setting.